### Import packages and prepare data

In [ ]:
# Import packages
# Basic
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

# Data analyze
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RepeatedKFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error

# Models
from sklearn.linear_model import LinearRegression, Lasso
import xgboost as xgb

In [ ]:
# Set some numbers
SEED = 12
METRIC = 'neg_root_mean_squared_error' #the main metric we will use for grid search and cross validation

In [ ]:
# Rread the dataset
df = pd.read_csv('Real_data.csv')

# Data preprocess
df.rename(columns={'ID_new': 'CID'}, inplace=True)
df = df[['CID', 'AMT', 'TIME', 'TD', 'LNDV']]
# Apply exponential function to 'LNDV' while keeping zeros unchanged
df['LNDV'] = df['LNDV'].apply(lambda x: np.exp(x) if x != 0 else 0)
df['NDV'] = df.groupby('CID')['LNDV'].shift(-1)
df = df.dropna(subset=['NDV'])

# Data normalization
df_old = df.copy()
scaler = StandardScaler()
df[['AMT', 'TD', 'LNDV', 'NDV']] = scaler.fit_transform(df[['AMT','TD', 'LNDV', 'NDV']])

In [ ]:
# Split data into training and test sets
cids = df['CID'].unique()
train_cids, test_cids = train_test_split(cids, test_size=0.3, random_state=SEED)
train_df = df[df['CID'].isin(train_cids)]
test_df = df[df['CID'].isin(test_cids)]

X_train = train_df[['TD', 'AMT', 'LNDV']]
y_train = train_df['NDV']
X_test = test_df[['TD', 'AMT', 'LNDV']]
y_test = test_df['NDV']

### Run ML methods to get the predictions

In [ ]:
# Define ML models
models = {
    'LR': LinearRegression(),
    'Lasso': Lasso(),
    'XGBoost': xgb.XGBRegressor(objective='reg:squarederror', seed=SEED)
}

In [ ]:
# Grid search to find the best hyperparameter set
param_grids = {
    'LR': {},
    'Lasso': {'alpha': [0.01, 0.1, 1]},
    'XGBoost': {'n_estimators': [50, 100, 300], 'max_depth': [3, 5, 10], 'learning_rate': [0.05, 0.1, 0.3]}
}

best_estimators = {}
for name, model in models.items():
    print(f"Grid search for {name}")
    grid_search = GridSearchCV(model, param_grids[name], cv=10, scoring=METRIC, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_estimators[name] = grid_search.best_estimator_
    print(f"Best parameters for {name}: {grid_search.best_params_}")

In [ ]:
def reverse_normalize_column(normalized_col, original_col):
    mean = original_col.mean()
    std = original_col.std()
    return normalized_col * std + mean

In [ ]:
# Fit models with best hyperparameters and evaluate
metrics = {}
preds = {}

for name, model in best_estimators.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    pred_real = reverse_normalize_column(pred, df_old['NDV'])
    preds[name] = pred_real
      
    metrics[name] = {
        'RMSE': root_mean_squared_error(y_test, pred),
        'MAE': mean_absolute_error(y_test, pred),
        'R^2': r2_score(y_test, pred)
    }
    print(f"Evaluation for {name}: RMSE={metrics[name]['RMSE']:.4f}, MAE={metrics[name]['MAE']:.4f}, R^2={metrics[name]['R^2']:.4f}")

### Get AUCs for evaluation

In [ ]:
pred_df = pd.DataFrame(preds)

# Reset the indices of both dataframes
test_data_reset = test_df.reset_index(drop=True)
pred_df_reset = pred_df.reset_index(drop=True)
# Concatenate the dataframes along axis 1
res = pd.concat([test_data_reset, pred_df_reset], axis=1)

In [ ]:
df_new = res.copy()

# Reverse normalization for 'AMT', 'TD', 'LNDV', and 'NDV'
columns_to_reverse = ['AMT', 'TD', 'LNDV', 'NDV']
for col in columns_to_reverse:
    df_new[col] = reverse_normalize_column(df_new[col], df_old[col])

# Apply exponential function to 'NDV', 'LR', 'Lasso', 'XGBoost'
df_new = df_new[["CID", "TIME","LNDV", "NDV","LR", "Lasso", "XGBoost"]]

In [ ]:
# Select a random patient (CID)
random_patient = np.random.choice(df_new['CID'].unique())

# Filter the data for the selected patient
patient_data = df_new[df_new['CID'] == random_patient]

# Extract Time and Concn columns for the real concentration curve
time = patient_data['TIME']
real_concn = patient_data['NDV']

# Extract predicted concentration columns for the predicted curves
predicted_columns = ['LR', 'Lasso', 'XGBoost']
patient_data.loc[patient_data.index[0], predicted_columns] = 0

# Initialize dictionary to store AUC values
auc_values = {}

# Plotting
plt.figure(figsize=(15, 10))

# Plot the real concentration curve
plt.subplot(2, 3, 1)
plt.plot(time, real_concn, label='Real Concentration', color='blue')
plt.title(f'Patient {random_patient} - Real Concentration')
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.legend()

# Calculate and plot AUC for the real concentration curve
auc_real = np.trapz(real_concn, time)
auc_values['Real'] = auc_real
plt.text(0.5, 0.8, f'AUC = {auc_real:.2f}', transform=plt.gca().transAxes, fontsize=10, verticalalignment='top')

# Plot the predicted concentration curves
for i, col in enumerate(predicted_columns, start=1):
    plt.subplot(2, 3, i+1)
    plt.plot(time, real_concn, label='Real Concentration', color='blue')
    plt.plot(time, patient_data[col], label=f'Predicted {col}', color='red', linestyle='--')
    plt.title(f'Patient {random_patient} - Predicted {col}')
    plt.xlabel('Time')
    plt.ylabel('Concentration')
    plt.legend()

    # Calculate and plot AUC for the predicted concentration curve
    auc_pred = np.trapz(patient_data[col], time)
    auc_values[col] = auc_pred
    plt.text(0.5, 0.8, f'AUC = {auc_pred:.2f}', transform=plt.gca().transAxes, fontsize=10, verticalalignment='top')

plt.tight_layout()
plt.show()